In [1]:
import os
import warnings
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from mne.io import read_raw_edf
import yasa
from preprocessing import crop_hypno, extract_features
import mne
import sleepecg
import random

In [2]:
# Define paths (can be defined in config files)
parent_dir = os.path.dirname(os.getcwd())
eeg_dir = parent_dir+'/data/edfs/shhs2/'
hypno_dir = parent_dir+'/data/annotations-events-profusion/shhs2/'
out_dir = '/output/features/'
if not os.path.isdir(parent_dir+out_dir):
    os.mkdir(parent_dir+out_dir)

In [3]:
df_subj = pd.read_csv(parent_dir+"/output/split/shhs_split.csv")
df_subj = df_subj.query("set == 'training'").set_index("subj")

In [4]:
print(df_subj.shape[0], 'subjects remaining')
df_subj.head(10)

80 subjects remaining


,age,male,bmi,ahi,ethnicity,set,hypertension
subj,,,,,,,
200077,41.0,1,23.388687,9.738220,caucasian,training,0.0
200078,54.0,1,30.211833,19.685039,caucasian,training,1.0
200079,56.0,0,35.451050,26.000000,other,training,0.0
200080,54.0,1,32.645673,12.450000,caucasian,training,0.0
200081,40.0,0,31.644286,2.632794,caucasian,training,0.0
200086,68.0,1,23.225432,22.258065,caucasian,training,0.0
200089,42.0,0,23.889873,6.885246,caucasian,training,0.0
200091,77.0,0,22.892820,7.407407,caucasian,training,1.0
200103,51.0,1,32.933601,30.334572,caucasian,training,0.0


In [5]:
col = ['ECG_meanNN','ECG_maxNN','ECG_minNN','ECG_rangeNN','ECG_SDNN','ECG_RMSSD','ECG_SDSD','ECG_NN50',
       'ECG_NN20','ECG_pNN50','ECG_pNN20','ECG_medianNN','ECG_madNN','ECG_iqrNN','ECG_cvNN',
       'ECG_cvSD','ECG_meanHR','ECG_maxHR', 'ECG_minHR', 'ECG_stdHR',
       'ECG_SD1', 'ECG_SD2', 'ECG_S', 'ECG_SD1_SD2_ratio', 'ECG_CSI', 'ECG_CVI','ECG_total_power', 
       'ECG_vlf', 'ECG_lf', 'ECG_lf_norm', 'ECG_hf', 'ECG_hf_norm', 'ECG_lf_hf_ratio']

In [6]:
df = []
include = ['EEG', 'EOG(L)', 'EMG','ECG']
sf = 100

for sub in tqdm(df_subj.index):
    eeg_file = eeg_dir + 'shhs2-' + str(sub) + '.edf'
    hypno_file = hypno_dir + 'shhs2-' + str(sub) + '-profusion.xml'
    
    # Check that file exists
    if not os.path.isfile(eeg_file):
        warnings.warn("File not found %s" % eeg_file)
        continue
    if not os.path.isfile(hypno_file):
        warnings.warn("File not found %s" % hypno_file)
        continue

    # LOAD EEG DATA
    try:
        raw = read_raw_edf(eeg_file, preload=False, verbose=0)
        raw.drop_channels(np.setdiff1d(raw.info['ch_names'], include))
        # Skip subjects if channel were not found
        assert len(raw.ch_names) == len(include)
        raw.load_data()
    except:
        continue
    _, times = raw[:]  
        
    # Resample and high-pass filter 
    raw.resample(sf, npad="auto")
    ecg = raw.get_data()[0]
    

    
    # LOAD HYPNOGRAM
    hypno, sf_hyp = yasa.load_profusion_hypno(hypno_file)
    # (Optional) We keep up to 15 minutes before / after sleep
    # hypno, tmin, tmax = crop_hypno(hypno)
    # raw.crop(tmin, tmax)
    # Check that hypno and data have the same number of epochs
    n_epochs = hypno.shape[0]
    if n_epochs != np.floor(raw.n_times / sf / 30):
        print("- Hypno and data size do not match.")
        continue
    
    # Convert hypnogram to str
    df_hypno = pd.Series(hypno)
    df_hypno.replace({0: 'W', 1: 'N1', 2: 'N2', 3: 'N3', 4: 'R'}, inplace=True)
    stage_min = df_hypno.value_counts(sort=False) / 2

    # INCLUSION CRITERIA (DISABLED)
    # Hypnogram must include all stages
#     if np.unique(hypno).tolist() != [0, 1, 2, 3, 4]:
#         print("- Not all stages are present.")
#         continue
#     # If the duration is not between 4 to 12 hours, skip subject
#     if not(4 < n_epochs / 120 < 12):
#         print("- Recording too short/long.")
#         continue

    # EXTRACT FEATURES
    features = extract_features(df_subj, sub, raw, include)
    # Add hypnogram
    features['stage'] = df_hypno.to_numpy()
    
    # ---------------------
    # extract ECG features
    # Read ECG data
    heartbeat_times = sleepecg.detect_heartbeats(ecg, sf)/sf
    sleep_stage_duration = 30
    # record_duration = times[-1]
    num_stages = features.shape[0]
    # check if ECG epoch number is the same with EEG epoch number
    if (features.shape[0] - num_stages) != 0:
        print('Skip due to different numbers of epoch')
        continue
    stage_times = np.arange(num_stages) * sleep_stage_duration
    min_rri, max_rri = None, None
    lookback, lookforward = 240, 60
    rri = sleepecg.preprocess_rri(
            np.diff(heartbeat_times),
            min_rri=min_rri,
            max_rri=max_rri,
        )
    rri_times = heartbeat_times[1:]
    ##
    fs_rri_resample = 100
    max_nans = 0.5
    feature_ids = []

    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        td_feature = sleepecg.feature_extraction._hrv_timedomain_features(rri,
                        rri_times,stage_times,lookback,lookforward,)
        fd_feature = sleepecg.feature_extraction._hrv_frequencydomain_features(rri,rri_times,
                    stage_times,lookback,lookforward,fs_rri_resample, max_nans, feature_ids)
    
    td_feat = pd.DataFrame(td_feature)
    fd_feat = pd.DataFrame(fd_feature)
    df_ecg = pd.concat([td_feat,fd_feat],axis = 1)
    df_ecg.columns = col
    features.reset_index(inplace=True)
    temp = pd.concat([features, df_ecg], axis=1)
    temp.set_index(['subj','epoch'])
    
    
    df.append(temp)

df = pd.concat(df).set_index(['subj','epoch'])

  0%|          | 0/80 [00:00<?, ?it/s]

In [7]:
# Convert to category
df['stage'] = df['stage'].astype('category')
df = df[df['stage'].isin(['N1', 'N2', 'N3', 'R', 'W'])]
df.stage = df.stage.cat.remove_categories(9)

In [8]:
# Export
df.to_parquet(parent_dir+out_dir + "features_with_ecg_100HZ_nsrr_shhs2.parquet")

In [9]:
df.dtypes.stage

CategoricalDtype(categories=['N1', 'N2', 'N3', 'R', 'W'], ordered=False)

## Stats

In [10]:
# different because we skip one with different epoch #
df['stage'].value_counts(normalize=True, sort=True)

W     0.426067
N2    0.329285
R     0.118415
N3    0.093290
N1    0.032943
Name: stage, dtype: float64

In [11]:
df.groupby('stage')['eeg_iqr'].median()

stage
N1    11.716578
N2    15.945243
N3    27.347612
R     11.046414
W     23.898228
Name: eeg_iqr, dtype: float32

In [12]:
df.groupby('stage')['ECG_meanNN'].median()

stage
N1    0.896657
N2    0.938594
N3    0.915854
R     0.924954
W     0.760785
Name: ECG_meanNN, dtype: float64

In [13]:
df.groupby('stage')['ECG_CSI'].median()

stage
N1    2.714922
N2    2.126074
N3    1.632590
R     2.686968
W     1.436331
Name: ECG_CSI, dtype: float64

In [14]:
df.groupby('stage')['ECG_SD1_SD2_ratio'].median()

stage
N1    0.368335
N2    0.470350
N3    0.612524
R     0.372167
W     0.696218
Name: ECG_SD1_SD2_ratio, dtype: float64